# XGBoost 
## playing nicely between distributed systems 

this post talks about distributing Pandas Dataframes with Dask and then handing them over to distributed XGBoost for training

More generally, it discusses the value of lauching multiple distributed systems in the same shared-memory processes and smoothly handing data back and forth between them. 

XGBoost is widely-used in Kaggle competitions, often to impressive effect. For larger datasets and/or faster training, XGBoost also comes with its own distributed computing system that lets it scale to multiple machines on a cluster. However, before we can use distributed XGBoost we need to do three things: 

1. Prepare and clean our data, most probably with lots of wrangling in Pandas. 
2. Set up XGBoost master and workers 
3. Hand data our cleaned data from a bunch of distributed Pandas dataframes to XGBoost workers across out cluster.

This ends up being surprisingly easy. This blogpost gives a quick example using `Dask.dataframe` to do distributed Pandas data wrangling, then uses the new `dask-xgboost` package to setup an XGBoost cluster inside the Dask cluster and perform the handoff. 

After this example we'll talk about the general design and what this means for other distributed systems

In [1]:
# code 

the `dask-xgboost` project is pretty small and simple. Given a Dask cluster of one central scheduler and several distributed workers, it starts up an XGBoost scheduler *in the same process* running the Dask scheduler and starts an XGBoost worker within each of the Dask workers. They share the same physical processes and memory spaces. Then we ask the Dask.dataframe to fully materialise in RAM and we ask where all of the constituent Pandas dataframes live. We tell each Dask worker to give all of the Pandasdataframes that it has to its local XGBoost worker and then just let XGBoost do its thing. Dask doesn't power XGBoost. it just sets it up, gives it data, and lets it do its work in the background. 

Because both Dask and XGBoost can live in the same Python process they can share bytes between each other without cost, can monitor each other, etc. These two distributed systems co-exist together in multiple processes with multiple systems in the same way that Numpy and Pandas operate together within a single process. Sharing distributed processes with multiple systems can be really beneficial if you want to use multiple specialised services easily and avoid large monolithic frameworks. 

Genreally, this serve other systems is how Dask operates when possible.  